In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# Define utility functions
def sma(window, columns):
    return np.sum(np.abs(window[columns]), axis=1).mean()

def energy(window, column):
    return np.sum(window[column]**2) / len(window[column])

def iqr(window, column):
    return np.percentile(window[column], 75) - np.percentile(window[column], 25)

def mad(window, column):
    return np.median(np.abs(window[column] - np.median(window[column])))

def meanFreq(window, column):
    fft_vals = np.fft.fft(window[column])
    freqs = np.fft.fftfreq(len(fft_vals))
    mag = np.abs(fft_vals)
    return np.sum(freqs * mag) / np.sum(mag)

def bandsEnergy(window, column, num_bins=64):
    fft_vals = np.fft.fft(window[column])
    mag = np.abs(fft_vals)[:num_bins]  # Use first num_bins for energy calculation
    return np.sum(mag**2) / num_bins

def angle(vec1, vec2):
    """Calculate the angle between two vectors"""
    dot_product = np.dot(vec1, vec2)
    magnitude = np.linalg.norm(vec1) * np.linalg.norm(vec2)
    return np.arccos(dot_product / magnitude) if magnitude != 0 else 0

# Function to process a single file and extract advanced features
def process_shaking_file(file_path, window_size=100, stride=50):
    data = pd.read_csv(file_path)
    data.columns = data.columns.str.strip()  # Clean column names

    windows = []
    for i in range(0, len(data) - window_size, stride):
        window = data.iloc[i:i + window_size]

        # Features dictionary for each window
        features = {}

        # Process each axis for Acceleration, Gyroscope, and Magnetometer
        for sensor in ['Acceleration', 'Gyro', 'MagneticField']:
            for axis in ['X', 'Y', 'Z']:
                col = f"{sensor}{axis}"
                features[f'mean_{sensor.lower()}_{axis.lower()}'] = window[col].mean()
                features[f'std_{sensor.lower()}_{axis.lower()}'] = window[col].std()
                features[f'mad_{sensor.lower()}_{axis.lower()}'] = mad(window, col)
                features[f'max_{sensor.lower()}_{axis.lower()}'] = window[col].max()
                features[f'min_{sensor.lower()}_{axis.lower()}'] = window[col].min()
                features[f'energy_{sensor.lower()}_{axis.lower()}'] = energy(window, col)
                features[f'bandsEnergy_{sensor.lower()}_{axis.lower()}'] = bandsEnergy(window, col)
                features[f'iqr_{sensor.lower()}_{axis.lower()}'] = iqr(window, col)
                features[f'meanFreq_{sensor.lower()}_{axis.lower()}'] = meanFreq(window, col)
                features[f'skewness_{sensor.lower()}_{axis.lower()}'] = skew(window[col])
                features[f'kurtosis_{sensor.lower()}_{axis.lower()}'] = kurtosis(window[col])

        # Add SMA (Signal Magnitude Area) feature
        features['sma_acc'] = sma(window, ['AccelerationX', 'AccelerationY', 'AccelerationZ'])
        features['sma_gyro'] = sma(window, ['GyroX', 'GyroY', 'GyroZ'])
        features['sma_mag'] = sma(window, ['MagneticFieldX', 'MagneticFieldY', 'MagneticFieldZ'])

        # Calculate angles
        acc_vector = [window['AccelerationX'].mean(), window['AccelerationY'].mean(), window['AccelerationZ'].mean()]
        gyro_vector = [window['GyroX'].mean(), window['GyroY'].mean(), window['GyroZ'].mean()]
        features['angle_acc_gyro'] = angle(acc_vector, gyro_vector)

        # Add label for the behavior
        features['label'] = 'shaking_roll'  # 'shaking' behavior

        windows.append(features)

    return pd.DataFrame(windows)

# List of uploaded files
file_paths = [
    'sensordata1.csv',
    'sensordata2.csv',
    'sensordata3.csv',
    'sensordata4.csv',
    'sensordata5.csv',
    'sensordata6.csv'
]

# Process all files and combine into a single DataFrame
window_size = 100
stride = 50
all_shaking_features = []

for file_path in file_paths:
    shaking_features = process_shaking_file(file_path, window_size, stride)
    all_shaking_features.append(shaking_features)

# Combine all processed features into a single DataFrame
shaking_features_df = pd.concat(all_shaking_features, ignore_index=True)

# Save the processed DataFrame to a CSV file for later use
output_file_path = "shaking_roll_features_advanced_processed.csv"
shaking_features_df.to_csv(output_file_path, index=False)

output_file_path